In [1]:
from dotenv import load_dotenv
from pathlib import Path
import sys


sys.path.append(Path("..").resolve().as_posix())
_ = load_dotenv()

In [2]:
from src.importers.video_importer import VideoImporter

/home/azureuser/.conda/envs/training_toolkit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
video_importer = VideoImporter()

dataset = video_importer(name="msrvtt-qa_1000", video_path="/data/msrvtt-qa/videos", annotation_path="/data/msrvtt-qa/qa.json")
dataset.save_to_disk("msrvtt-qa_1000")

Saving the dataset (1/1 shards): 100%|██████████| 1001/1001 [00:00<00:00, 273589.10 examples/s]


In [4]:
from src.model_presets import llava_next_video_preset
from src.data_presets import local_video_preset

In [5]:
from src.train_builder import build_trainer

- add templated cache dir to avoid collisions
- add dataset name field into the uhh... somewhere
- add generic data preset factory to allow for custom datasets

In [6]:
trainer = build_trainer(
    **local_video_preset.as_kwargs(),
    **llava_next_video_preset.as_kwargs(),
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


800
201


You are using a model of type llava_next to instantiate a model of type llava_next_video. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.88s/it]


In [ ]:
trainer.train()